# XGBoost model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use('seaborn')

# Data preparation

In [ ]:
data = pd.read_csv('../../data/SamDysch_glucose_2-5-2022.csv', skiprows=[0])
data.index = pd.to_datetime(data['Device Timestamp'], format="%d-%m-%Y %H:%M")

In [ ]:
# drop non-historic glucose records
data = data[data['Record Type'] == 0]

# only keep bg
to_keep = [
    'Historic Glucose mmol/L',
]
data = data[to_keep]

data = data.rename(columns={'Historic Glucose mmol/L': 'reading'})

data.head()

In [ ]:
# drop NaNs
data = data.dropna()

# Setup hypo threshold

In [ ]:
HYPO_THRESHOLD = 3.9
data['is_hypo'] = (data['reading'] < HYPO_THRESHOLD).astype(int)

In [ ]:
# adding some time variables
data['hour'] = data.index.hour
data['day'] = data.index.dayofweek
data['month'] = data.index.month

# OneHotEncode hours

In [ ]:
data = pd.get_dummies(data, prefix='hour', columns=['hour'])
print(data.columns)

# creating a lagged and rolling variables
* Was I hypo 15 mins ago? 30 mins ago? Etc
* Rolling average of last N readings
* Sign of gradient of last N readings:
    * I.e., is BG rising, falling, or stable?
    
## Lagged features

In [ ]:
# create lags
# To ensure that we do not make a lag between periods of sensor non-usage, create a new df with the lagged indices & merge onto original data frame
def create_lag(df, lag):
    tolerance = 15 * lag
    freq = '15min'
    print(f'Creating lag of {tolerance} minutes')
    lagged_copy = df[['reading']].shift(lag, freq=freq)
    lagged_copy.rename(columns={'reading': f'lagged_reading_{lag}'}, inplace=True)
    
    merged = pd.merge_asof(df, lagged_copy, left_index=True, right_index=True, direction='backward', tolerance=pd.Timedelta(minutes=tolerance))
    # merged = pd.merge_asof(copy, lagged_copy, left_index=True, right_index=True, direction='backward')
    return merged

NLAGS = 8
for lag in range(1, NLAGS):
    data = create_lag(data, lag)

In [ ]:
# For ease of variable calculation, drop the nans
data = data.dropna()

In [ ]:
# lagged hypo bools
for lag in range(1, NLAGS):
    data[f'is_lagged_hypo_{lag}'] = (data[f'lagged_reading_{lag}'] < HYPO_THRESHOLD).astype(int)

## Rolling features

In [ ]:
# simple differences of lags - was reading higher, lower, or stable?
for lag in range(2, NLAGS):
    data[f'diff_{lag}'] = data['lagged_reading_1'] - data[f'lagged_reading_{lag}']

# gradients - how quick is BG changing?
interval = 15
for lag in range(2, NLAGS):
    data[f'rate_{lag}'] = data[f'diff_{lag}'] / (interval * lag)

## train, test, validation split

In [ ]:
TRAIN_SPLIT = 0.65
VAL_SPLIT = 0.2
TEST_SPLIT = 0.15

In [ ]:
itrain = int(TRAIN_SPLIT * len(data))
ival = int(VAL_SPLIT * len(data))
itest = int(TEST_SPLIT * len(data))

train_data = data.iloc[:itrain]
val_data = data.iloc[itrain:itrain + ival]
test_data = data.iloc[itrain + ival:]

# Variable selection

In [ ]:
rates_and_diffs = [f'diff_{v}' for v in range(2, NLAGS)]
rates_and_diffs.extend([f'rate_{v}' for v in range(2, NLAGS)])

# to fairly compare with baseline, drop any historical variables with time delta < 45 mins
vars_to_drop = [
    'month',
    'day',
    'reading',
    'is_lagged_hypo_1',
    'is_lagged_hypo_2',
    'lagged_reading_1',
    'lagged_reading_2',
]
vars_to_drop.extend(rates_and_diffs)

train_data = train_data.drop(vars_to_drop, axis='columns')
val_data = val_data.drop(vars_to_drop, axis='columns')
test_data = test_data.drop(vars_to_drop, axis='columns')

print(train_data.columns)
print(val_data.columns)
print(test_data.columns)

In [ ]:
target = 'is_hypo'

X_train = train_data.drop([target], axis='columns')
y_train = train_data[target]

X_val = val_data.drop(target, axis='columns')
y_val = val_data[target]

X_test = test_data.drop(target, axis='columns')
y_test = test_data[target]

print(X_train.columns)

In [ ]:
# redefine train = train + validation for cross validation
X_train = pd.concat([X_train, X_val])
y_train = pd.concat([y_train, y_val])

In [ ]:
# class weights
xgb_weight = float(y_train[y_train == 0].count()) / y_train[y_train == 1].count()
print(xgb_weight)

# objective function setup

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
from xgboost import XGBClassifier
import optuna

def objective(trial):
    
    # hyperparameter space
    n_estimators = trial.suggest_int('n_estimators', 10, 300)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    eta = trial.suggest_categorical('eta', [0.1, 0.15, 0.2, 0.3])
    gamma = trial.suggest_int('gamma', 0, 5)

    
    # define model
    model = XGBClassifier(
        #verbosity=2,
        n_estimators=n_estimators,
        eta=eta,
        gamma=gamma,
        max_depth=max_depth,
        reg_lambda=1,
        reg_alpha=0,
        subsample=0.5,
        scale_pos_weight=xgb_weight,
        objective='binary:logistic'   
    )
    
    # fit and evaluate model
    ftwo_scorer = make_scorer(fbeta_score, beta=2)
    splits = TimeSeriesSplit(n_splits=5)
    scores = cross_val_score(model, X_train, y_train, cv=splits, scoring=ftwo_scorer)
    
    return np.mean(scores)

# run optuna trials

In [ ]:
# optimise
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=400)

In [ ]:
# print(f'Best trial: {study.best_trial}')
print(f'Best value: {study.best_value}')
print(f'Best parameters: {study.best_params}')

# fitting optimised model

In [ ]:
# define model
from xgboost import XGBClassifier
model = XGBClassifier(
    n_estimators=108,
    eta=0.15,
    gamma=0,
    max_depth=4,
    reg_lambda=1,
    reg_alpha=0,
    subsample=0.5,
    objective='binary:logistic'   
)

# fit model
model.fit(
    X_train,
    y_train,
    eval_metric=['aucpr', 'logloss'],
    eval_set=[(X_train, y_train), (X_test, y_test)],
    #verbose=True
    verbose=False
)

In [ ]:
results = model.evals_result()

fig, ax = plt.subplots(1, 2, figsize=(20, 7))

ax[0].plot(results['validation_0']['aucpr'], label='Train')
ax[0].plot(results['validation_1']['aucpr'], label='Test')
ax[0].legend(loc='best')
ax[0].set_ylabel('AUC (PR)')

ax[1].plot(results['validation_0']['logloss'], label='Train')
ax[1].plot(results['validation_1']['logloss'], label='Test')
ax[1].legend(loc='best')
ax[1].set_ylabel('logloss')


fig.show()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, fbeta_score
y_pred = model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1: {f1_score(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2)}')

cm = confusion_matrix(y_test, y_pred, normalize='all')
sns.heatmap(cm, annot=True, square=True)

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve
y_pred_prob = model.predict_proba(X_test)[:, 1]

# fpr, tpr, _ = roc_curve(y_test, y_pred)
# precision, recall, _ = precision_recall_curve(y_test, y_pred)

fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

ax[0].plot(fpr, tpr, 'b')
ax[0].plot([0, 1], [0, 1], 'r--')
ax[0].set_ylabel('True Positive Rate')
ax[0].set_xlabel('False Positive Rate')

ax[1].plot(recall, precision, 'b')
ax[1].set_xlabel('Recall')
ax[1].set_ylabel('Precision')

plt.show()
roc_auc = auc(fpr, tpr)
print(roc_auc)